In [ ]:
## FUNCTION 4
#This function checks a specific company (identified by its EHRAID) acquirers.
def check_acquirers(ehraid, df=True):
    
    '''
    
    This function takes in the EHRAID of a company and returns a list of dictionaries with information about the companies that took over the given company.

    Parameters:
    ehraid (str): EHRAID of the company to check the acquirers for.
    df (bool): True by default. If True, returns the data as a pandas DataFrame. If False, returns the data as a list of dictionaries.

    Returns:
    if df=True (default):
    acquirers (dataframe): a dataframe with information about the companies that took over the given company. If df=False, returns a list of dictionaries.

    if df=False:
    acquirers (list): a list of dictionaries with information about the companies that took over the given company.

    '''

    if type(ehraid) != str:
        return "Please provide a string of the EHRAID of the company as input. Example: '110662'"

    company_url = f'https://www.zefix.ch/ZefixREST/api/v1/firm/{ehraid}'
    r = requests.get(company_url)
    data = r.json()

    if r.status_code != 200:
        print(f"Failed to fetch details for EHRAID {ehraid}:", r.text)
        return None
    
    was_taken_over_by = data.get('wasTakenOverBy', [])
    if not was_taken_over_by:
        print(f"No acquisition records found for EHRAID {ehraid}.")
        return None
    
    # get the legal form names from the API
    legal_form_url = "https://www.zefix.ch/ZefixREST/api/v1/legalForm"
    legal_forms_response = requests.get(legal_form_url)
    legal_forms = legal_forms_response.json()

    if legal_forms_response.status_code != 200:
        print("Failed to fetch details for legal forms. The legal form will be provided as a numeric id. Read the zefix documentation for further details on what that id means.")
    
    
    # search the details for the acquirer company and save them
    acquirers = []
    for acquirer in was_taken_over_by:
        acquirer_details = requests.get(f'https://www.zefix.ch/ZefixREST/api/v1/firm/{acquirer["ehraid"]}')
        acquirer_data = acquirer_details.json()
        
        if acquirer_details.status_code != 200:
            print(f"Failed to fetch details for acquirer EHRAID {acquirer['ehraid']}:", acquirer_details.text)
            continue

        # Here we specify the information we are looking for

        # create a dictionary with legal form id as key
        legal_forms_dictionary = {item['id']: item for item in legal_forms}

        # get the English name of a legal form associated with the given ID provided by the API
        firm_legal_form_name = legal_forms_dictionary[data['legalFormId']]['name']['en']

        record = {
            'name': acquirer_data['name'],
            'ehraid': acquirer_data['ehraid'],
            'legalSeat': acquirer_data.get('legalSeat'),
            "legalFormId": acquirer_data.get("legalFormId") if legal_forms_response.status_code != 200 else firm_legal_form_name ,
            'status': acquirer_data.get('status'),
            "cantonalExcerptWeb": acquirer_data.get("cantonalExcerptWeb"),
            'deleteDate': acquirer_data.get('deleteDate'),
            'purpose':acquirer_data.get('purpose'),
            'capitalNominal':acquirer_data.get('capitalNominal'),
            }
        
        #translate the information in 'purpuse' to english
        if record["purpose"]: 
            translator = Translator()
            purpose_english = translator.translate(record['purpose'], dest='en')
            record['purpose'] = purpose_english.text

        #translate the information in 'status' to english
        if record["status"]:    
            if record['status'].upper() == 'EXISTIEREND':
                record['status'] = 'Active'
            elif record['status'].upper() == 'GELOESCHT':
                record['status'] = 'Deleted'
            elif record['status'].upper() == 'IN_AUFLOESUNG':
                record['status'] = 'In Liquidation'
            else:
                translator = Translator()
                status_english = translator.translate(record['status'], dest='en')
                record['status'] = status_english.text

        acquirers.append(record)
         
    if df==True:
        acquirers = pd.DataFrame(acquirers)

    return acquirers

In [ ]:
##EXAMPLE FUNCTION 4
# Exemple of EHRAID(UBS)
ehraid = "110662"
acquired_details_df = check_acquirers(ehraid, df=True)

acquired_details_df

In [ ]:
get_acquisitions_data('421132', df=True)